In [2]:
from shiny import *
import pandas as pd
import numpy as np
import plotly.graph_objects as go


df = pd.read_csv('./transaction_data.csv', encoding='utf-8')

print(df.tail())

df['date'] = pd.to_datetime(df['date'])
df['order_id'] = df['order_id'].astype(str)

print(df.info())



ModuleNotFoundError: No module named 'shiny'

Notes:
- Date column and order_id were the wrong d type so i adjusted accordingly


In [ ]:
shipment_agg = df.groupby('order_id').agg(
    lines = ('order_id', 'size'),
    qty = ('quantity', 'sum'),
    date = ('date','max')
).reset_index(drop = True)

daily_activity = shipment_agg.groupby('date').agg(
    daily_orders = ('date','size'),
    daily_lines = ('lines','sum'),
    daily_qty = ('qty','sum')
).reset_index()

daily_activity.head(100)

,date,daily_orders,daily_lines,daily_qty
0,2021-01-04,2764,4091,12912
1,2021-01-05,2750,4397,9300
2,2021-01-06,2768,4070,9908
3,2021-01-07,2493,4008,7573
4,2021-01-08,2342,3987,10087
...,...,...,...,...
95,2021-04-24,199,275,629
96,2021-04-26,4862,7238,9846
97,2021-04-27,3280,5575,9288
98,2021-04-28,3521,5819,10740


Notes:

- modified tabular structure to aggregate total number of daily orders, lines and quantity
- added a new column to display the total number of daily orders, lines and quantity


In [ ]:
def plot_daily_activity(daily_activity):
    # Create traces for each metric
    trace1 = go.Scatter(x=daily_activity['date'], y=daily_activity['daily_orders'],
                        mode='lines+markers', name='Daily Orders',
                        text=daily_activity.apply(lambda row: f"Date: {row['date']}<br>Daily Orders: {row['daily_orders']}<br>Daily Lines: {row['daily_lines']}<br>Daily Pieces: {row['daily_qty']}", axis=1),
                        hoverinfo='text')
    trace2 = go.Scatter(x=daily_activity['date'], y=daily_activity['daily_lines'],
                        mode='lines+markers', name='Daily Lines',
                        text=daily_activity.apply(lambda row: f"Date: {row['date']}<br>Daily Orders: {row['daily_orders']}<br>Daily Lines: {row['daily_lines']}<br>Daily Pieces: {row['daily_qty']}", axis=1),
                        hoverinfo='text')
    trace3 = go.Scatter(x=daily_activity['date'], y=daily_activity['daily_qty'],
                        mode='lines+markers', name='Daily Pieces',
                        text=daily_activity.apply(lambda row: f"Date: {row['date']}<br>Daily Orders: {row['daily_orders']}<br>Daily Lines: {row['daily_lines']}<br>Daily Pieces: {row['daily_qty']}", axis=1),
                        hoverinfo='text')

    # Create figure and add traces
    fig = go.Figure()
    fig.add_trace(trace1)
    fig.add_trace(trace2)
    fig.add_trace(trace3)

    # Update layout
    fig.update_layout(title='Daily Activity for Year: 2023',
                      xaxis_title='Date',
                      yaxis_title='Total Orders/Lines/Pieces',
                      legend_title='Metric',
                      hovermode='closest')

    return fig

In [ ]:
plot_daily_activity(daily_activity)

In [ ]:
sku_activity_profile =  (
    df
    .groupby('sku_id')
    .agg(
        lines = ('sku_id','size'),
        qty = ('quantity','sum')
    )
    .reset_index()
    .sort_values(by = ['lines','qty'],
                 ascending = [False, False])
)
# Calculate percent of total and running percent for lines
total_lines = sku_activity_profile['lines'].sum()
sku_activity_profile['percent_of_total_lines'] = sku_activity_profile['lines'] / total_lines
sku_activity_profile['running_percent_lines'] = sku_activity_profile['percent_of_total_lines'].cumsum()

# Calculate percent of total and running percent for qty
total_qty = sku_activity_profile['qty'].sum()
sku_activity_profile['percent_of_total_qty'] = sku_activity_profile['qty'] / total_qty
sku_activity_profile['running_percent_qty'] = sku_activity_profile['percent_of_total_qty'].cumsum()

sku_activity_profile.head()

,sku_id,lines,qty,percent_of_total_lines,running_percent_lines,percent_of_total_qty,running_percent_qty
6531,SPB0100,69021,86837,0.050850,0.050850,0.023960,0.023960
2478,108-4065,66569,75840,0.049043,0.099893,0.020926,0.044886
2411,02-003-001-01,46867,50511,0.034528,0.134421,0.013937,0.058823
5357,HLY-BRANDEDMAILER,44078,44078,0.032473,0.166895,0.012162,0.070986
6344,REF 0006-U1,35975,36447,0.026504,0.193398,0.010057,0.081042


In [ ]:
# Determine "Class - Lines" based on the running percent of lines
sku_activity_profile['class - lines'] = pd.cut(sku_activity_profile['running_percent_lines'],
                                                    bins=[0, 0.4, 0.6, 0.8, 1],
                                                    labels=['F+', 'F', 'M', 'S'],
                                                    right=True,
                                                    include_lowest=True)

    # Determine "Class - Units" based on the running percent of qty
sku_activity_profile['class - units'] = pd.cut(sku_activity_profile['running_percent_qty'],
                                                   bins=[0, 0.4, 0.6, 0.8, 1],
                                                   labels=['A+', 'A', 'B', 'C'],
                                                   right=True,
                                                   include_lowest=True)

    # Perform final sorting by 'lines' in descending order for the output
final_output = sku_activity_profile.sort_values(by='lines', ascending=False)

    # Select and order columns with the updated names and new classifications
final_output = final_output[['sku_id', 'lines', 'qty', 'running_percent_lines', 'percent_of_total_lines', 'Class - Lines', 'running_percent_qty', 'percent_of_total_qty', 'Class - Units']]
final_output.head(50)


,sku_id,lines,qty,running_percent_lines,percent_of_total_lines,Class - Lines,running_percent_qty,percent_of_total_qty,Class - Units
6531,SPB0100,69021,86837,0.050850,0.050850,F+,0.023960,0.023960,A+
2478,108-4065,66569,75840,0.099893,0.049043,F+,0.044886,0.020926,A+
2411,02-003-001-01,46867,50511,0.134421,0.034528,F+,0.058823,0.013937,A+
5357,HLY-BRANDEDMAILER,44078,44078,0.166895,0.032473,F+,0.070986,0.012162,A+
6344,REF 0006-U1,35975,36447,0.193398,0.026504,F+,0.081042,0.010057,A+
5358,HLY-DROPCARD,30228,46170,0.215668,0.022270,F+,0.093782,0.012739,A+
4499,DUP020WNA,22010,22141,0.231884,0.016215,F+,0.099891,0.006109,A+
6392,SDG0100,16099,19567,0.243744,0.011861,F+,0.105290,0.005399,A+
6405,SGX0100,12943,14841,0.253280,0.009535,F+,0.109385,0.004095,A+
6687,SVT0100,11596,18830,0.261823,0.008543,F+,0.114580,0.005196,A+
